# Configurations
API access is only available for on-premise deployments.
In order to interact with the Medical Chatbot via API you need:
*   the URL of the API endpoint
*   a valid API key - an admin or developer user can create that for you from the Chat UI.




In [23]:
base_URL = "https://chat.staging.chatbot.johnsnowlabs.dev/api/v1"
#base_URL = "https://chat.johnsnowlabs.com/api/v1"

In [26]:
from google.colab import files

# Function to upload file
uploaded = files.upload()

# Read the content of the uploaded file into a variable x
for filename in uploaded.keys():
    with open(filename, 'r') as file:
        APIKey = file.read()


Saving APIKey.txt to APIKey (2).txt


In [27]:
import requests
import json

headers = {
  "Authorization": "Bearer "+ APIKey
}

# Available Tools
The Medical Chatbot is equipped with several tools, each tailored to meet different information needs and research requirements. These tools are:
* The **Medical LLM** - Access to text generation features tuned for the medical domain.
* **MedResearch** - Up-to-Date Medical Knowledge Base containing research results published as part of PubMed, bioRxiv, medRxiv, and many more.
* **Web Search** - Search the web for pages relevant to a given query, which John will then use as references to generate a response. For optimal results, use concise queries.
* **Drug Insights** -Learn about drugs, their interactions and side effects.
* Healthcare **NLP Tools** - Access to several state-of-the-art accuracy tools, each designed for specific tasks related to medical data handling and analysis.

They collectively provide a comprehensive solution for a wide range of queries.

In [28]:
url = base_URL + "/tools"

response = requests.get(url, headers=headers)

output = [{'name': func['function']['name'], 'description': func['function']['description']} for func in response.json()['data']]
print(json.dumps(output, indent=2))

[
  {
    "name": "medical_research",
    "description": "Up-to-Date Medical Knowledge Base containing research results published as part of PubMed, bioRxiv, medRxiv, and many more."
  },
  {
    "name": "search_web",
    "description": "Search the web for pages relevant to a given query, which will then be used as references to generate a response. For optimal results, use concise queries."
  },
  {
    "name": "deidentification",
    "description": "Identify and mask sensitive information (PHI) in patient notes or unstructured medical text with over 99% accuracy."
  },
  {
    "name": "obfuscation",
    "description": "Obfuscate sensitive information (PHI) in patient notes or medical text to safeguard patient privacy and adhere to regulatory requirements."
  },
  {
    "name": "ner_medical",
    "description": "Extract medical entities or terms from user-provided text effortlessly with no arguments needed."
  },
  {
    "name": "ner_medical_oncology",
    "description": "Extract onco

# Get the list of available KBs

In [29]:
import requests
import json

url = base_URL + "/knowledge-bases"

response = requests.get(url, headers=headers)

output = [{'description': kb['name']} for kb in response.json()['data']]
print(json.dumps(output, indent=2))


[
  {
    "description": "Drugleaflet"
  },
  {
    "description": "MDPI"
  },
  {
    "description": "PubMed"
  },
  {
    "description": "medRxiv"
  },
  {
    "description": "bioRxiv"
  },
  {
    "description": "Semantic Scholar"
  },
  {
    "description": "Cigna Insurance Plans"
  }
]


# Medical Text Summarization using the Medical LLM
The Medical Chatbot can distill complex content into concise, digestible summaries. It uses advanced language processing to extract key points from lengthy medical texts, research papers, and clinical guidelines, presenting them in a clear and succinct format.
You can prompt it to respond according to your needs.

For instance:
*   Write a 3 sentence summary of the following text:
*   Write a short summary of the patient's note by highlighting the main complaint and the suggested treatment.




In [30]:
sample_note_cardio = '''Medical Specialty:Cardiovascular / Pulmonary

Sample Name: Acute Inferior Myocardial Infarction

Description: Patient presents with a chief complaint of chest pain admitted to Coronary Care Unit due to acute inferior myocardial infarction.

(Medical Transcription Sample Report)

CHIEF COMPLAINT:  Chest pain.HISTORY OF PRESENT ILLNESS: The patient is a 40-year-old white male who presents with a chief complaint of "chest pain".The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. The severity of the pain has progressively increased. He describes the pain as a sharp and heavy pain which radiates to his neck & left arm. He ranks the pain a 7 on a scale of 1-10. He admits some shortness of breath & diaphoresis. He states that he has had nausea & 3 episodes of vomiting tonight. He denies any fever or chills. He admits prior episodes of similar pain prior to his PTCA in 1995. He states the pain is somewhat worse with walking and seems to be relieved with rest. There is no change in pain with positioning. He states that he took 3 nitroglycerin tablets sublingually over the past 1 hour, which he states has partially relieved his pain. The patient ranks his present pain a 4 on a scale of 1-10. The most recent episode of pain has lasted one-hour.The patient denies any history of recent surgery, head trauma, recent stroke, abnormal bleeding such as blood in urine or stool or nosebleed.REVIEW OF SYSTEMS: All other systems reviewed & are negative.
PAST MEDICAL HISTORY: Diabetes mellitus type II, hypertension, coronary artery disease, atrial fibrillation, status post PTCA in 1995 by Dr. Smith .SOCIAL HISTORY:  Denies alcohol or drugs. Smokes 2 packs of cigarettes per day. Works as a banker.FAMILY HISTORY:  Positive for coronary artery disease (father & brother).MEDICATIONS:  Aspirin 81 milligrams QDay. Humulin N. insulin 50 units in a.m. HCTZ 50 mg QDay. Nitroglycerin 1/150 sublingually PRN chest pain.ALLERGIES:  Penicillin.
PHYSICAL EXAM:  The patient is a 40-year-old white male.General: The patient is moderately obese but he is otherwise well developed & well nourished. He appears in moderate discomfort but there is no evidence of distress. He is alert, and oriented to person place and circumstance. There is no evidence of respiratory distress. The patient ambulates without gait abnormality or difficulty.HEENT: Normocephalic/atraumatic head. Pupils are 2.5 mm, equal round and react to light bilaterally. Extra-ocular muscles are intact bilaterally. External auditory canals are clear bilaterally. Tympanic membranes are clear and intact bilaterally.Neck: No JVD. Neck is supple. There is free range of motion & no tenderness, thyromegaly or lymphadenopathy noted.Pharynx: Clear, no erythema, exudates or tonsillar enlargement.Chest: No chest wall tenderness to palpation. Lungs: Clear to auscultation bilaterally. Heart: irregularly-irregular rate and rhythm no murmurs gallops or rubs. Normal PMIAbdomen: Soft, non-distended. No tenderness noted. No CVAT.Skin: Warm, diaphoretic, mucous membranes moist, normal turgor, no rash noted.Extremities: No gross visible deformity, free range of motion. No edema or cyanosis. No calf/ thigh tenderness or swelling.
COURSE IN EMERGENCY DEPARTMENT:  The patient's chest pain improved after the sublingual nitroglycerine and completely resolved with the Nitroglycerin Drip at 30 ug/Minute. He tolerated the TPA well. He was transferred to the CCU in a stable conditionPROCEDURES:10:40 PM Dr. Smith (cardiologist) apprised. He agrees with T PA per 90 minute protocol & IV nitroglycerin drip. He is to come see patient in the emergency department.10:45 PM risks & benefits of TPA discussed with patient & his family. They agree with administration of TPA and are willing to accept the risks.10:50 PM TPA started.11:20 PM Dr. Smith present in emergency department assisting with patient care.DIAGNOSTIC STUDIES:CBC: WBC 14.2, hematocrit 33.5, platelets 316
Chem 7: Na 142, potassium 4.5, chloride 102, CO2 22.6, BUN 15, creatinine 1.2, glucose 186Serum Troponin I: 2.5Chest x-ray: Lung fields clear. No cardiomegaly or other acute findingsEKG: Atrial fibrillation with Ventricular rate of 65. Acute inferior ischemic changes noted i.e. ST elevation III & aVF (refer to EKG multimedia).Cardiac monitor: Sinus rhythm-atrial of fibrillation rate 60s-70s.TREATMENT:Heparin lock X. 2.Nasal cannula oxygen 3 liters/minute.Aspirin 5 grains chew & swallow.
Nitroglycerin drip at 30 micrograms/minute.Cardiac monitor.TPA 90 minute protocol.Heparin IV 5000 unit bolus followed by 1000 units/hour.IMPRESSION:  Acute Inferior Myocardial Infarction.PLAN:  Patient admitted to Coronary Care Unit under the care of Dr. Smith.'''

In [31]:
import requests

url = base_URL + "/chat/completions"

data = {
  "messages": [
    {
      "role": "user",
      "content": "Write a short summary of the following patient's note by highlighting the main complaint, diagnosis and the suggested treatment:\n\n --- \n\n" + sample_note_cardio
    }
  ]
}

response = requests.post(url, headers=headers, json=data)
print(response.json()['content'])

**Summary of Patient's Note**

**Main Complaint:** Chest pain

**Diagnosis:** Acute Inferior Myocardial Infarction (heart attack)

**Treatment:**

* Sublingual nitroglycerin and nitroglycerin drip at 30 micrograms/minute to relieve chest pain
* Tissue Plasminogen Activator (TPA) administered according to 90-minute protocol to dissolve blood clots
* Heparin IV 5000 unit bolus followed by 1000 units/hour to prevent further clotting
* Aspirin 5 grains chew and swallow to prevent platelet aggregation
* Cardiac monitor to track heart rhythm
* Nasal cannula oxygen 3 liters/minute to increase oxygen supply

This response can be enhanced with specialized tools like Medical Research assistant, which provide verified citations and references. Feel free to specify if you'd like to use this tool.


# Generate a response with history
The medical chatbot understands context and can manage follow-up questions, ensuring that healthcare professionals can engage in detailed dialogues to extract specific information necessary for clinical decisions or research inquiries. Whether discussing symptoms, treatments, or the latest medical research, the chatbot’s conversational capabilities make it an effective tool for dynamic and informative exchanges.

In [32]:
import requests

url = base_URL + "/chat/completions"

data = {
  "messages": [
    {
      "role": "user",
      "content": "What is the drug prescribed for hypothyroidism?"
    },
    {
      "role": "assistant",
      "content": "The drug prescribed for hypothyroidism is Levothyroxine sodium tablets, also referred to as T4. It is used as a replacement therapy in primary, secondary, and tertiary congenital or acquired hypothyroidism."
    },
    {
      "role": "user",
      "content": "Can I take it while on Iron Supplements?"
    }
  ],
    "tools": [
         "medical_research"
        ]

}

response = requests.post(url, headers=headers, json=data)
response.json()

{'status': 'success',
 'tool': 'medical_research',
 'content': '**Interactions between Levothyroxine Sodium and Iron Supplements**\n\nLevothyroxine sodium, a common treatment for hypothyroidism, can interact with iron supplements, affecting the absorption and efficacy of both medications [Document 2, Document 4]. Studies have shown that concomitant intake of iron salt supplements and levothyroxine can impair the absorption of levothyroxine, particularly when taken in tablet form [Document 2]. However, new levothyroxine formulations, such as liquid solutions, may be less affected by iron supplements [Document 2].\n\n**Mechanism of Interaction**\n\nThe stomach plays a crucial role in the absorption of levothyroxine, and factors such as gastric juice pH, volume, and viscosity can influence the dissolution of levothyroxine from its solid form [Document 5]. Iron supplements can alter the gastric environment, potentially affecting levothyroxine absorption.\n\n**Clinical Implications**\n\nTo 

# Use tools/agents with a selection of KBs

In [33]:
import requests

url = base_URL + "/chat/completions"

data = {
  "messages": [
    {
      "role": "user",
      "content": "What is vitamin A?"
    }
  ],
    "tools": [[
         "medical_research",
         { "knowledge_bases": ["dataset-bgs-pubmed"]}
        ]]

}

response = requests.post(url, headers=headers, json=data)
response.json()

{'status': 'success',
 'tool': 'medical_research',
 'content': '**Introduction**\n\nVitamin A is an essential nutrient that plays a crucial role in various bodily functions, including vision, immune function, and cell growth and differentiation [Document 2]. It is a fat-soluble vitamin that can be obtained through dietary sources, such as preformed retinol, retinyl esters, and provitamin A carotenoids [Document 4].\n\n**Primary Functions of Vitamin A**\n\nVitamin A is distributed within the body to support chromophore synthesis in the eyes and retinoid signaling in most other tissues [Document 1]. The active metabolite of vitamin A, retinoic acid (RA), is involved in various biological processes, including cell growth and differentiation, embryonic development, and organogenesis [Document 5]. RA signaling is highly dose-dependent, and a fine-tuning regulatory mechanism is required to maintain appropriate levels of RA [Document 5].\n\n**Consequences of Vitamin A Deficiency**\n\nVitamin 

# Medical question answering with automatic tool selection

When all tools are enabled, the Medical Chatbot will automatically choose the one to use for answering the question asked by the user.

In [34]:
import requests

url = base_URL + "/chat/completions"

data = {
  "messages": [
    {
      "role": "user",
      "content": "What is the best treatment for diabetes?"
    }
  ],
  "tools": [
    "medical_research",
    "search_web",
    "deidentification",
    "obfuscation",
    "ner_medical",
    "ner_medical_oncology",
    "ner_medical_posology"
  ]
}

response = requests.post(url, headers=headers, json=data)
response.json()

{'status': 'success',
 'tool': 'medical_research',
 'content': '**Current Treatment Options for Diabetes**\n\nThe treatment of diabetes has evolved significantly in recent years, with a focus on individualized approaches and the use of newer glucose-lowering agents [Document 3]. The primary aim of diabetes management is to prevent complications, which are a major cause of morbidity and mortality [Document 6]. A multifactorial approach, focusing on diet, exercise, glucose levels, blood pressure, and lipids, is the most effective strategy for preventing complications [Document 6].\n\n**Pharmacological Therapy**\n\nPharmacological therapy is a crucial component of diabetes management, with various classes of medications available, including metformin, sodium-glucose cotransporter 2 (SGLT2) inhibitors, glucagon-like peptide 1 (GLP1) receptor agonists, and dipeptidyl peptidase 4 (DPP4) inhibitors [Document 3]. The choice of medication depends on individual patient characteristics, such as t

# Using Clinical Advisor

In [35]:
import requests

input_prompt = '''Create a neat and correctly apportioned table outlining the Comprehensive Clinical Determination Capabilities in Functional Medicine, that you can assist with by using your LLM and Clinical Advisor tools when I ask you about my patients or research including:

Functional Medicine Approach
Root Cause Analysis
DIGIN Model Application
Personalized Nutrition
Lifestyle & Environmental Evaluations
Systems Biology
Patient-Centered Care
Core Imbalance
Therapeutic Options
Preventive Measures
Provide comprehensive explanations of complex physiological and biochemical processes, supported by citations from reputable sources. Include analogies that accurately convey intricate concepts in an accessible manner, without sacrificing scientific rigor. Offer practical applications and actionable insights for clinicians, enabling effective communication with patients. Assume a dialogue with fellow healthcare experts and practitioners, leveraging a doctorate-level background in functional medicine. Prioritize thoroughness, nuance, and sophistication in your responses, ensuring they meet the standards of an expert audience.'''

url = base_URL + "/chat/completions"

data = {
  "messages": [
    {
      "role": "user",
      "content": input_prompt
    }
  ],
  "tools": [
    "clinical_advisor"
  ]
}

response = requests.post(url, headers=headers, json=data)
response.json()

{'status': 'success',
 'tool': 'clinical_advisor',
 'content': "**Comprehensive Clinical Determination Capabilities in Functional Medicine**\n\n| **Capability** | **Description** | **Key Components** |\n| --- | --- | --- |\n| **Functional Medicine Approach** | A patient-centered, systems biology approach to healthcare that focuses on identifying and addressing the underlying causes of disease [Document 1]. | Holistic, dynamic, integrative, cross-disciplinary approach to biological complexity |\n| **Root Cause Analysis** | A method used to identify the underlying causes of a problem or disease, rather than just treating its symptoms [Document 4]. | Identifying and addressing the root causes of disease, rather than just treating symptoms |\n| **DIGIN Model Application** | A framework for understanding the complex interactions between diet, gut health, inflammation, and lifestyle factors in the development of chronic disease [Document 7]. | Diet, gut health, inflammation, lifestyle factor

# Web Search
Web search integration allows the chatbot to access the most current information available on the internet, including news, scientific research, and other dynamically changing data. This is particularly crucial in fields like medicine, where new research findings and medical guidelines frequently emerge.

In [36]:
import requests

url = base_URL + "/chat/completions"

data = {
  "messages": [
    {
      "role" : "user",
      # "content": "What are the drugs approved by FDA in June and July 2024 and what do they treat?"
      "content" : "Is ozempic available in Romania?"
    }
  ],
  "tools": [
    "search_web"
  ]
}

response = requests.post(url, headers=headers, json=data)
response.json()

{'status': 'success',
 'tool': 'search_web',
 'content': 'According to the article from Stirile Pro TV, Ozempic was available in Romania but will only be available until July 31st, due to commercial reasons. However, since the current date is April 6, 2025, it appears that Ozempic is no longer available in Romania.',
 'references': [{'title': 'Ozempic | European Medicines Agency (EMA)',
   'label': 'Ozempic | European Medicines Agency (EMA)',
   'url': 'https://www.ema.europa.eu/en/medicines/human/EPAR/ozempic',
   'content': 'This is a summary of the European public assessment report (EPAR) for Ozempic. It explains how the Agency assessed the medicine to recommend its authorisation ...'},
  {'title': 'Ozempic drug trafficking: An entire network exposed in Romania',
   'label': 'Ozempic drug trafficking: An entire network exposed in Romania',
   'url': 'https://www.newsendip.com/ozempic-drug-trafficking-network-romania/',
   'content': 'A trafficking network in Romania has been uncover

# Using Medical Agents
The Medical Chatbot is equipped with several medical agents, each tailored to meet different information needs and research requirements. These tools are:


|Tool Name                    |Description                                                                                                                                                 |
|-----------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------|
|oncology                     |Extract oncological entities and their relations from clinical documentation including EHRs, medical transcripts, and oncology notes.                       |
|posology                     |Extract Drug, Dosage, Duration, Form, Frequency, Route, and Strength entities and their relations from patient notes.                                       |
|adverse_drug_event           |Identify adverse reactions of drugs (ADE) from reviews, tweets, and medical text.                                                                           |
|radiology                    |Identify tests, imaging techniques, and findings within radiology reports, establish their assertion statuses and their relations.                          |
|social_determinants_of_health|Identify socio-environmental health determinants like access to care, diet, employment, and housing from health records.                                    |
|voice_of_the_patient         |Identify clinical and behavioral entities from patient-generated content, including tweets, blogs, forums, and other social media posts.                    |
|medical                      |Extract medical entities or terms from user-provided text effortlessly with no arguments needed.                                                            |
|deidentification             |Identify and mask sensitive information (PHI) in patient notes or unstructured medical text with over 99% accuracy.                                         |
|obfuscation                  |Obfuscate sensitive information (PHI) in patient notes or medical text to safeguard patient privacy and adhere to regulatory requirements.                  |
                                                         |

To force the Medical Chatbot to use a specific tool add it as unique option on the tools list int the request sent to completions endpoint.


## Clinical De-identification

In [37]:
phi_text = '''
Record date: 2069-04-07
Mr. Villegas is seen today.  I have not seen him since November.
About three weeks ago he stopped his Prednisone on his own because
he was gaining weight.  He does feel that his shoulders are
definitely improved.
On examination today, BP 120/80.  His joint examination is much
improved with better ROM of the shoulders and no peripheral joint
synovitis.
Clinical Impression:
1:  Inflammatory arthritis - possibly RA - with response noted to
Hydroxychloroquine along with Prednisone.  He has stopped the
Prednisone, and I would not restart it yet.
2:  New onset of symptoms suspicious for right-sided carotid
disease.  Will arrange for carotid ultrasound studies.  Patient
advised to call me if he develops any worsening symptoms.
Xzavian G. Tavares, M.D.
XGT:holmes
DD: 04/07/69
DT: 04/15/69
DV: 04/07/69
'''

In [38]:
import requests

url = base_URL + "/chat/completions"

data = {
  "messages": [
    {
      "role": "user",
      "content": phi_text
    }
  ],
  "tools": [
    "deidentification"
  ]
}

response = requests.post(url, headers=headers, json=data)
output = response.json()
print(output['content'])

## Extract oncological entities from patient notes

In [39]:
oncology_note ='''Sample Name: Lung Cancer & MI - Hospice Cosult

Description: Patient with metastatic non-small-cell lung cancer, on hospice with inferior ST-elevation MI. The patient from prior strokes has expressive aphasia, is not able to express herself in a clear meaningful fashion.


REASON FOR CONSULT:  I was asked to see this patient with metastatic non-small-cell lung cancer, on hospice with inferior ST-elevation MI.HISTORY OF PRESENT ILLNESS:  The patient from prior strokes has expressive aphasia, is not able to express herself in a clear meaningful fashion. Her daughter who accompanies her is very attentive whom I had met previously during drainage of a malignant hemorrhagic pericardial effusion last month. The patient has been feeling well for the last several weeks, per the daughter, but today per the personal aide, became agitated and uncomfortable at about 2:30 p.m. At about 7 p.m., the patient began vomiting, was noted to be short of breath by her daughter with garbled speech, arms flopping, and irregular head movements. Her daughter called 911 and her symptoms seemed to improve. Then, she began vomiting. When the patient's daughter asked her if she had chest pain, the patient said yes.She came to the emergency room, an EKG showed inferior ST-elevation MI. I was called immediately and knowing her history, especially, her hospice status with recent hemorrhagic pericardial effusion, I felt thrombolytic was contraindicated and she would not be a candidate for aggressive interventional therapy with PCI/CABG. She was begun after discussion with the oncologist, on heparin drip and has received morphine, nitro, and beta-blocker, and currently states that she is pain free. Repeat EKG shows normalization of her ST elevation in the inferior leads as well as normalization of prior reciprocal changes.PAST MEDICAL HISTORY:  Significant for metastatic non-small-cell lung cancer. In early-to-mid December, she had an admission and was found to have a malignant pericardial effusion with tamponade requiring urgent drainage. We did repeat an echo several weeks later and that did not show any recurrence of the pericardial effusion. She is on hospice from the medical history, atrial fibrillation, hypertension, history of multiple CVA.MEDICATIONS:  Medications as an outpatient:1. Amiodarone 200 mg once a day.
2. Roxanol concentrate 5 mg three hours p.r.n. pain.ALLERGIES: CODEINE. NO SHRIMP, SEAFOOD, OR DYE ALLERGY.FAMILY HISTORY:  Negative for cardiac disease.SOCIAL HISTORY:  She does not smoke cigarettes. She uses alcohol. No use of illicit drugs. She is divorced and lives with her daughter. She is a retired medical librarian from Florida.REVIEW OF SYSTEMS: Unable to be obtained due to the patient's aphasia.
PHYSICAL EXAMINATION:  Height 5 feet 3, weight of 106 pounds, temperature 97.1 degrees, blood pressure ranges from 138/82 to 111/87, pulse 61, respiratory rate 22. O2 saturation 100%. On general exam, she is an elderly woman with now marked aphasia, which per her daughter waxes and wanes, was more pronounced and she nods her head up and down when she says the word, no, and conversely, she nods her head side-to-side when she uses the word yes with some discordance in her head gestures with vocalization. HEENT shows the cranium is normocephalic and atraumatic. She has dry mucosal membrane. She now has a right facial droop, which per her daughter is new. Neck veins are not distended. No carotid bruits visible. Skin: Warm, well perfused. Lungs are clear to auscultation anteriorly. No wheezes. Cardiac exam: S1, S2, regular rate. No significant murmurs. PMI is nondisplaced. Abdomen: Soft, nondistended. Extremities: Without edema, on limited exam. Neurological exam seems to show only the right facial droop.DIAGNOSTIC/LABORATORY DATA:  EKGs as reviewed above. Her last ECG shows normalization of prior ST elevation in the inferior leads with Q waves and first-degree AV block, PR interval 280 milliseconds. Further lab shows sodium 135, potassium 4.2, chloride 98, bicarbonate 26, BUN 9, creatinine 0.8, glucose 162, troponin 0.17, INR 1.27, white blood cell count 1.3, hematocrit 31, platelet count of 179.Chest x-ray, no significant pericardial effusion.IMPRESSION:  The patient is a 69-year-old woman with metastatic non-small-cell lung cancer with a recent hemorrhagic pericardial effusion, now admitted with cerebrovascular accident and transient inferior myocardial infarction, which appears to be canalized. I will discuss this in detail with the patient and her daughter, and clearly, her situation is quite guarded with likely poor prognosis, which they are understanding of.RECOMMENDATIONS:1. I think it is reasonable to continue heparin, but clearly she would be at risk for hemorrhagic pericardial effusion recurrence.
2. Morphine is appropriate, especially for preload reduction and other comfort measures as appropriate.3. Would avoid other blood thinners including Plavix, Integrilin, and certainly, she is not a candidate for a thrombolytic with which the patient and her daughter are in agreement with after a long discussion.Other management as per the medical service. I have discussed the case with Dr. X of the hospitalist service who will be admitting the patient.

'''

In [40]:
import requests

url = base_URL + "/chat/completions"

data = {
  "messages": [
    {
      "role": "user",
      "content": oncology_note
    }
  ],
  "tools": [
    "ner_medical_oncology"
  ]
}

response = requests.post(url, headers=headers, json=data)
output = response.json()


In [41]:
from IPython.core.display import display, HTML
display(HTML(output['content']))

Extract radiology entities from patient notes

In [42]:
radiology_note ='''Medical Specialty:Radiology
Description: Left heart cath, selective coronary angiogram, right common femoral angiogram, and StarClose closure of right common femoral artery.

EXAM:  Left heart cath, selective coronary angiogram, right common femoral angiogram, and StarClose closure of right common femoral artery.REASON FOR EXAM:  Abnormal stress test and episode of shortness of breath.PROCEDURE:  Right common femoral artery, 6-French sheath, JL4, JR4, and pigtail catheters were used.FINDINGS:1. Left main is a large-caliber vessel. It is angiographically free of disease,2. LAD is a large-caliber vessel. It gives rise to two diagonals and septal perforator. It erupts around the apex. LAD shows an area of 60% to 70% stenosis probably in its mid portion. The lesion is a type A finishing before the takeoff of diagonal 1. The rest of the vessel is angiographically free of disease.3. Diagonal 1 and diagonal 2 are angiographically free of disease.
4. Left circumflex is a small-to-moderate caliber vessel, gives rise to 1 OM. It is angiographically free of disease.5. OM-1 is angiographically free of disease.6. RCA is a large, dominant vessel, gives rise to conus, RV marginal, PDA and one PL. RCA has a tortuous course and it has a 30% to 40% stenosis in its proximal portion.7. LVEDP is measured 40 mmHg.8. No gradient between LV and aorta is noted.Due to contrast concern due to renal function, no LV gram was performed.Following this, right common femoral angiogram was performed followed by StarClose closure of the right common femoral artery.
IMPRESSION:1. 60% to 70% mid left anterior descending stenosis.2. Mild 30% to 40% stenosis of the proximal right coronary artery.3. Status post StarClose closure of the right common femoral artery.PLAN: Plan will be to perform elective PCI of the mid LAD.
'''

In [43]:
import requests

url = base_URL + "/chat/completions"

data = {
  "messages": [
    {
      "role": "user",
      "content": radiology_note
    }
  ],
  "tools": [
    "extract_medical_radiology"
  ]
}

response = requests.post(url, headers=headers, json=data)
output = response.json()
from IPython.core.display import display, HTML
display(HTML(output['content']))

## Extract Drug Information


In [44]:
posology = '''The patient is Jane Park, a 30-year-old female with a long history of diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. Jane did receive a course of Bactrim for 14 days for UTI. Evidently, at some point in time, the patient was noted to develop a pressure-type wound on the sole of her left foot and left great toe. She was also noted to have a large sacral wound; this is in a similar location with previous laminectomy, and this continues to receive daily care. The patient was transferred secondary to inability to participate in full physical and occupational therapy and continue medical management of her diabetes, the sacral decubitus, left foot pressure wound, and associated complications of diabetes. She is given Fragmin 5000 units subcutaneously daily, OxyContin 30 mg p.o. q.12 h., folic acid 1 mg daily, levothyroxine 0.1 mg p.o. , Prevacid 30 mg daily, Avandia 4 mg daily, Norvasc 10 mg daily, Lexapro 20 mg daily, aspirin 81 mg daily, Senna 2 tablets p.o. q.a.m., Neurontin 400 mg p.o. t.i.d., magnesium citrate 1 bottle p.o , Wellbutrin 100 mg p.o. daily, and Bactrim DS b.i.d.'''


In [45]:
import requests

url = base_URL + "/chat/completions"

data = {
  "messages": [
    {
      "role": "user",
      "content": posology
    }
  ],
  "tools": [
    "ner_medical_posology"
  ]
}

response = requests.post(url, headers=headers, json=data)
output = response.json()


In [46]:
display(HTML(output['content']))